In [1]:
from datetime import datetime, timedelta
import json
from pymlb_statsapi import api


In [ ]:
def jPrint(obj: dict | list) -> None:
    print(json.dumps(obj, indent=2))


def getTeamId(teamName: str, season: int = None) -> int:
    seasonYear = season or datetime.now().year
    response = api.Team.teams(sportId=1, season=seasonYear)
    teamsData = response.json().get("teams", [])
    teamLookup = []
    for team in teamsData:
        namesList = [
            team["name"].lower(),
            team["teamName"].lower(),
            team["abbreviation"].lower(),
            team["shortName"].lower(),
            team["locationName"].lower(),
        ]
        teamLookup.append(
            {
                "id": team["id"],
                "name": team["name"],
                "namesList": namesList,
            }
        )
    for team in teamLookup:
        if teamName.lower() in team["namesList"]:
            return team["id"]
    return None


In [ ]:
def getTeamRoster(teamName: str, season: int = None) -> list:
    teamId = getTeamId(teamName, season)
    if not teamId:
        return []
    year = season or datetime.now().year
    response = api.Team.roster(teamId=teamId, season=year, rosterType="active").json()
    rosterData = response.get("roster", [])
    return rosterData


In [ ]:
bravesRoster = []
for player in getTeamRoster("Braves"):
    bravesRoster.append(
        {
            "id": player["person"]["id"],
            "name": player["person"]["fullName"],
            "jerseyNumber": player["jerseyNumber"],
            "position": player["position"]["abbreviation"],
        }
    )

jPrint(bravesRoster)


[
  {
    "id": 700363,
    "name": "AJ Smith-Shawver",
    "jerseyNumber": "32",
    "position": "P"
  },
  {
    "id": 607481,
    "name": "Aaron Bummer",
    "jerseyNumber": "49",
    "position": "P"
  },
  {
    "id": 663586,
    "name": "Austin Riley",
    "jerseyNumber": "27",
    "position": "3B"
  },
  {
    "id": 693821,
    "name": "Bryce Elder",
    "jerseyNumber": "55",
    "position": "P"
  },
  {
    "id": 519242,
    "name": "Chris Sale",
    "jerseyNumber": "51",
    "position": "P"
  },
  {
    "id": 664849,
    "name": "Danny Young",
    "jerseyNumber": "",
    "position": "P"
  },
  {
    "id": 678226,
    "name": "Daysbel Hern\u00e1ndez",
    "jerseyNumber": "62",
    "position": "P"
  },
  {
    "id": 800311,
    "name": "Didier Fuentes",
    "jerseyNumber": "75",
    "position": "P"
  },
  {
    "id": 686948,
    "name": "Drake Baldwin",
    "jerseyNumber": "30",
    "position": "C"
  },
  {
    "id": 689266,
    "name": "Dylan Dodd",
    "jerseyNumber": "46",
   

In [ ]:
for player in bravesRoster:
    print(f"{player['name']} (#{player['jerseyNumber']}) - {player['position']}")


AJ Smith-Shawver (#32) - P
Aaron Bummer (#49) - P
Austin Riley (#27) - 3B
Bryce Elder (#55) - P
Chris Sale (#51) - P
Danny Young (#) - P
Daysbel Hernández (#62) - P
Didier Fuentes (#75) - P
Drake Baldwin (#30) - C
Dylan Dodd (#46) - P
Dylan Lee (#52) - P
Eli White (#36) - RF
George Soriano (#) - P
Grant Holmes (#66) - P
Ha-Seong Kim (#7) - SS
Hayden Harris (#79) - P
Hunter Stratton (#65) - P
Hurston Waldrep (#64) - P
Ian Hamilton (#) - P
Jhancarlos Lara (#81) - P
Joe Jiménez (#77) - P
Joel Payamps (#) - P
Joey Wentz (#33) - P
Jorge Mateo (#) - SS
Jurickson Profar (#17) - LF
Matt Olson (#28) - 1B
Mauricio Dubón (#) - 2B
Michael Harris II (#23) - CF
Mike Yastrzemski (#18) - RF
Nacho Alvarez Jr. (#24) - SS
Ozzie Albies (#1) - 2B
Raisel Iglesias (#26) - P
Reynaldo López (#40) - P
Robert Suarez (#75) - P
Rolddy Muñoz (#70) - P
Ronald Acuña Jr. (#13) - RF
Sean Murphy (#12) - C
Spencer Schwellenbach (#56) - P
Spencer Strider (#99) - P
Tyler Kinley (#) - P


#### Examples That Return Formatted/Print Results...

In [ ]:
def printTeamTotalSeasonWins(teamName: str, season: int = None) -> None:
    teamId = getTeamId(teamName)
    year = season or datetime.now().year
    isCurrSeason = year == datetime.now().year

    response = api.Schedule.schedule(
        sportId=1, teamId=teamId, startDate=f"{year}-01-01", endDate=f"{year}-12-31"
    ).json()

    gameDates = response.get("dates", [])

    totalWins = 0
    for games in gameDates:
        for game in games.get("games", []):
            teams = game.get("teams", {})
            homeTeamResult = {
                "id": teams.get("home", {}).get("team", {}).get("id"),
                "isWinner": teams.get("home", {}).get("isWinner", False),
            }
            awayTeamResult = {
                "id": teams.get("away", {}).get("team", {}).get("id"),
                "isWinner": teams.get("away", {}).get("isWinner", False),
            }
            winningTeamId = (
                homeTeamResult["id"]
                if homeTeamResult["isWinner"]
                else awayTeamResult["id"] if awayTeamResult["isWinner"] else None
            )

            if winningTeamId == teamId:
                totalWins += 1

    print(
        f"The {teamName}{' have won ' if isCurrSeason else ' won '}{totalWins} games {'this season' if isCurrSeason else f'in {year}'}."
    )


In [ ]:
printTeamTotalSeasonWins("Braves", 2021)
printTeamTotalSeasonWins("Braves", 2023)
printTeamTotalSeasonWins("Braves", 2025)
printTeamTotalSeasonWins("Braves")


The Braves won 114 games in 2021.
The Braves won 124 games in 2023.
The Braves won 90 games in 2025.
The Braves have won 0 games this season.


#### SOME MORE COOL EXAMPLES TO TRY AND CONVERT TO USING THIS NEW API...


In [ ]:
# # Example: Print the linescores of all games won by the Atlanta Braves in July 2025
# for x in [
#     y
#     for y in statsapi.schedule(team=144, start_date="07/01/2025", end_date="07/31/2025")
#     if y.get("winning_team", "") == "Atlanta Braves"
# ]:
#     print(
#         "%s\nWinner: %s, Loser: %s\n%s\n\n"
#         % (
#             x["game_date"],
#             x["winning_team"],
#             x["losing_team"],
#             statsapi.linescore(x["game_id"]),
#         )
#     )


In [ ]:
# # Example: Print the linescores of all games won by the Atlanta Braves in July 2025
# for x in [
#     y
#     for y in statsapi.schedule(team=144, start_date="07/01/2025", end_date="07/31/2025")
#     if y.get("winning_team", "") == "Atlanta Braves"
# ]:
#     print(
#         "%s\nWinner: %s, Loser: %s\n%s\n\n"
#         % (
#             x["game_date"],
#             x["winning_team"],
#             x["losing_team"],
#             statsapi.linescore(x["game_id"]),
#         )
#     )

# # Example: Print the Braves 40-man Roster on opening day of the 2025 season
# print(
#     "Braves 40-man roster on opening day of the 2025 season:\n%s"
#     % statsapi.roster(
#         144,
#         "40Man",
#         date=statsapi.get("season", {"seasonId": 2025, "sportId": 1})["seasons"][0][
#             "regularSeasonStartDate"
#         ],
#     )
# )

# # Example: Print boxscore and linescore from Braves most recent game (which may be in progress or may not have started yet based on MLB response to 'last game' request)
# most_recent_game_id = statsapi.last_game(144)
# print(
#     statsapi.boxscore(most_recent_game_id),
#     statsapi.linescore(most_recent_game_id),
#     sep="\n\n",
# )

# # Other Examples:
# # - find team with longest name
# longest_team_name = max(
#     [
#         x["name"]
#         for x in statsapi.get(
#             "teams", {"sportIds": 1, "activeStatus": "Yes", "fields": "teams,name"}
#         )["teams"]
#     ],
#     key=len,
# )
# print(
#     "The team with the longest name is %s, at %s characters."
#     % (longest_team_name, len(longest_team_name))
# )

# # - print standings from July 4, 2025
# print(statsapi.standings(date="07/04/2025"))

# #  - print top 5 team leaders in walks for 2025 Braves
# print(statsapi.team_leaders(144, "walks", limit=5, season=2025))

# # - print top 10 all time career leaders in doubles
# print(
#     statsapi.league_leaders("doubles", statGroup="hitting", statType="career", limit=10)
# )

# # - print Ronald Acuña Jr.'s career hitting stats
# print(
#     statsapi.player_stats(
#         statsapi.lookup_player("Ronald Acuña Jr.")[0]["id"],
#         "hitting",
#         "career",
#     )
# )

# # - print list of scoring plays from 6/23/2024 Braves @ Yankees
# print(statsapi.game_scoring_plays(745727))
